In [1]:
import itertools
import nibabel as nb
import numpy as np
import pandas as pd
from tqdm import tqdm
from atlasreader import atlasreader as ar
from nilearn._utils import check_niimg

The Python package you are importing, AtlasReader, is licensed under the
BSD-3 license; however, the atlases it uses are separately licensed under more
restrictive frameworks.
By using AtlasReader, you agree to abide by the license terms of the
individual atlases. Information on these terms can be found online at:
https://github.com/miykael/atlasreader/tree/master/atlasreader/data



Relabel voxels in Yeo's canonical functional network maps to AAL atlas
---
## 1.1 First load voxel data from Yeo:

In [18]:
# For Yeo 17 Networks:
# Read Yeo labels from file, fixing spacing and new/empty lines
Yeo_labels = open('../data/atlases/Yeo/Yeo2011_7Networks_MNI152.txt', 'r')
Yeo_labels = Yeo_labels.read().split('\n')
#Yeo_labels = [Yeo_labels[i][17:] for i in range(len(Yeo_labels)-1)]
Yeo_labels.insert(0,"None") #I believe the areas were numbered 1-7 with "0" reserved for white matter

#Read Yeo matrix
Yeo_template = "../data/atlases/Yeo/Yeo2011_7Networks_MNI152.nii"
Yeo = nb.load(Yeo_template)
Yeo_data = Yeo.get_fdata()

### 1.2 Loop through MNI-152 template:

In [19]:
AAL_data = ar.get_atlas('aal')['image'].get_fdata()

AAL_atlas = ar.get_atlas('aal')

We would like to loop over the pixels and compare them between atlases but, unfortunately, the matrices are cropped to remove large regions full of zeros so the matrices are not 256x256x256 anymore.

We can try doing the same operation in real, xyz space instead of in MNI-type indices. We do so by using `coord_ijk_to_xyz` function in `atlasreader`.

In [20]:
Yeo_affine = check_niimg(Yeo_template).affine
AAL_affine = check_niimg(ar.get_atlas('aal')['image']).affine

### 1.3 Compute the AAL voxel-based distribution of networks per region (very slow)

Try using multiprocessing and see if it speeds up the process:

In [21]:
for label in Yeo_labels:
    print(label)

None
  1     Visual
  2     Somatomotor
  3     Dorsal_Attention
  4     Ventral_Attention
  5     Limbic
  6     Frontoparietal
  7     Default



In [48]:
from multiprocessing import Pool

nprocesses = 4 # number of parallel workers

# pre-allocate the dictionaries, we will append to these as we loop through the data:
AAL_dict_voxels = {}
AAL_dict_counts = {}

with Pool(processes=nprocesses) as pool:
    for n in tqdm(ar.get_atlas('aal')['labels']['name'], desc = 'pre-allocation'):
        # this creates a dict for each region name in AAL atlas
        # each region's dict will contain network labels or voxelcounts
        # Yeo's networks are numbered, so we append the integer number labels to each network name
        AAL_dict_counts.update({n: {Yeo_label:0 for Yeo_label in Yeo_labels}})
        # this is the voxels for each network
        AAL_dict_voxels.update({n: {Yeo_label:[] for Yeo_label in Yeo_labels}})

    for aal_i in tqdm(range(0, AAL_data.shape[0]), desc = 'Outter Loop'):
        for aal_j in range(0, AAL_data.shape[1]):
            for aal_k in range(0, AAL_data.shape[2]):
                # Get each iterated voxel's labels:
                AAL_voxel_label = AAL_data[aal_i, aal_j, aal_k]
                AAL_voxel_region = ar.get_label(AAL_atlas, AAL_voxel_label)
                
                if AAL_voxel_region != 'no_label':
                    # Excluding the unknown voxels, we re-do the affine conversion
                    # then assign the Yeo labels/voxels to our AAL dict
                    xyz = ar.coord_ijk_to_xyz(AAL_affine, [aal_i, aal_j, aal_k])
                    Yeo_ijk = ar.coord_xyz_to_ijk(Yeo_affine, xyz)[0]
                    
                    Yeo_voxel_label = int(Yeo_data[Yeo_ijk[0], Yeo_ijk[1], Yeo_ijk[2]])
                    Yeo_voxel_region = Yeo_labels[Yeo_voxel_label]
                    AAL_dict_counts[AAL_voxel_region][Yeo_voxel_region] += 1 # add one to voxel count
                    AAL_dict_voxels[AAL_voxel_region][Yeo_voxel_region].append(xyz[0]) # label region


Outter Loop: 100%|██████████| 75/75 [09:05<00:00,  7.27s/it]


## 2. Prepare the data

### 2.1 Make DataFrame with Network x ROI data
We still need to convert this to a user-friendly data frame after updating the voxel counts and voxel labels:

In [85]:
# get list of region names to work with our previous dictionary
AAL_region_names = ar.get_atlas('aal')['labels']['name']

# fill in the previous dictionary per region
AAL_dict_regions = {}
for region in AAL_region_names:
    AAL_dict_regions.update({region:AAL_dict_counts[region]})

# transform into Pandas Dataframe:
AAL_dict_counts = pd.DataFrame(AAL_dict_regions)
AAL_dict_counts

,Precentral_L,Precentral_R,Frontal_Sup_2_L,Frontal_Sup_2_R,Frontal_Mid_2_L,Frontal_Mid_2_R,Frontal_Inf_Oper_L,Frontal_Inf_Oper_R,Frontal_Inf_Tri_L,Frontal_Inf_Tri_R,...,Cerebelum_10_L,Cerebelum_10_R,Vermis_1_2,Vermis_3,Vermis_4_5,Vermis_6,Vermis_7,Vermis_8,Vermis_9,Vermis_10
None,881,916,1233,1117,1013,931,248,319,570,683,...,144,159,53,228,657,365,194,243,174,112
1 Visual,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,6,0,0,0,0
2 Somatomotor,1319,1629,108,336,0,26,4,4,0,0,...,0,0,0,0,0,0,0,0,0,0
3 Dorsal_Attention,811,442,428,477,126,119,50,275,0,29,...,0,0,0,0,0,0,0,0,0,0
4 Ventral_Attention,105,228,312,240,411,376,271,314,153,245,...,0,0,0,0,0,0,0,0,0,0
5 Limbic,0,0,99,99,29,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6 Frontoparietal,332,166,682,1455,1853,3207,316,472,1124,915,...,0,0,0,0,0,0,0,0,0,0
7 Default,78,0,2011,1403,1077,197,149,15,682,279,...,0,0,0,0,4,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2.2 Normalize each region.

In [86]:
AAL_df_normalized = AAL_dict_counts.div(AAL_dict_counts.sum(axis=0), axis = 1)
AAL_df_normalized

,Precentral_L,Precentral_R,Frontal_Sup_2_L,Frontal_Sup_2_R,Frontal_Mid_2_L,Frontal_Mid_2_R,Frontal_Inf_Oper_L,Frontal_Inf_Oper_R,Frontal_Inf_Tri_L,Frontal_Inf_Tri_R,...,Cerebelum_10_L,Cerebelum_10_R,Vermis_1_2,Vermis_3,Vermis_4_5,Vermis_6,Vermis_7,Vermis_8,Vermis_9,Vermis_10
None,0.249858,0.270926,0.253027,0.217866,0.224662,0.191564,0.238921,0.228020,0.225386,0.317527,...,1.0,1.0,1.0,1.0,0.987970,0.983827,1.0,1.0,1.0,1.0
1 Visual,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.006015,0.016173,0.0,0.0,0.0,0.0
2 Somatomotor,0.374078,0.481810,0.022163,0.065535,0.000000,0.005350,0.003854,0.002859,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3 Dorsal_Attention,0.230006,0.130731,0.087831,0.093037,0.027944,0.024486,0.048170,0.196569,0.000000,0.013482,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4 Ventral_Attention,0.029779,0.067436,0.064026,0.046811,0.091151,0.077366,0.261079,0.224446,0.060498,0.113901,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
5 Limbic,0.000000,0.000000,0.020316,0.019310,0.006432,0.000823,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
6 Frontoparietal,0.094158,0.049098,0.139955,0.283792,0.410956,0.659877,0.304432,0.337384,0.444444,0.425384,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
7 Default,0.022121,0.000000,0.412682,0.273649,0.238856,0.040535,0.143545,0.010722,0.269672,0.129707,...,0.0,0.0,0.0,0.0,0.006015,0.000000,0.0,0.0,0.0,0.0
,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


### Save to file

In [87]:
AAL_df_normalized.to_csv("AAL_dictionary_normalized.csv")

In [89]:
AAL_dict_counts.to_csv("AAL_dictionary_counts.csv")